In [20]:
from agents import Agent, trace, Runner, function_tool
from dotenv import load_dotenv
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

## agent Workflow



In [3]:
instruction1 = " You are asales agent working for ComplAI, \
    A COMPANY THAT prvoides a saas tool for ensuring soc2 compliance and preparing for audits. powered by ai. \
        you wirte a professional, serious cold email"

In [4]:
instruction1

' You are asales agent working for ComplAI,     A COMPANY THAT prvoides a saas tool for ensuring soc2 compliance and preparing for audits. powered by ai.         you wirte a professional, serious cold email'

In [ ]:
sales_agent = Agent(
    name = "Professional sales Agent",
    instructions = instruction1,
    model = "gpt-4o-mini"
)

In [6]:
sales_agent


Agent(name='Professional sales Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions=' You are asales agent working for ComplAI,     A COMPANY THAT prvoides a saas tool for ensuring soc2 compliance and preparing for audits. powered by ai.         you wirte a professional, serious cold email', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [13]:
result = Runner.run_streamed(sales_agent, "write a cold sales email ") # starts the agent , this run_Stremed tells agent to write the cold sales

async for event in result.stream_events(): # get events one by one gives a small poevces of ouput instesas od waole email
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):  # pick only the text poeces 
        print(event.data.delta, end = '', flush=True) # print each chunk immediately , so the email appears words ny words 


# is just a way to stream the AI’s response live instead of waiting for the full answer at the end.

Subject: Streamline Your SOC 2 Compliance with ComplAI

Hi [Recipient's Name],

I hope this message finds you well.

As your organization continues to navigate the complexities of data security and compliance, I wanted to introduce you to ComplAI—a powerful SaaS tool designed specifically to simplify SOC 2 compliance and prepare your team for audits.

Our AI-driven platform offers:

- **Automated Documentation**: Generate and maintain necessary documentation effortlessly.
- **Real-Time Monitoring**: Stay compliant with continuous monitoring and alerts.
- **Seamless Integration**: Connect with your existing tools for a streamlined experience.
- **Expert Guidance**: Access resources and expertise to guide you through audits.

With ComplAI, you can reduce the time and effort spent on compliance tasks, allowing your team to focus on what truly matters—growing your business.

I’d love to schedule a brief call to discuss how ComplAI can support your compliance goals. Are you available for a 

In [10]:
result


RunResultStreaming(input='write a cold sales email ', new_items=[], raw_responses=[], final_output=None, input_guardrail_results=[], output_guardrail_results=[], context_wrapper=RunContextWrapper(context=None, usage=Usage(requests=0, input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=0)), current_agent=Agent(name='Professional sales Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions=' You are asales agent working for ComplAI,     A COMPANY THAT prvoides a saas tool for ensuring soc2 compliance and preparing for audits. powered by ai.         you wirte a professional, serious cold email', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbo

In [14]:
sales_picker = Agent(
    name = " select the best sales cold email",
    instructions = " You pick the best cold sales email from the given options. \
    Imagine you are a customer and pick the one you are most likely to respond to. \
    Do not give an explanation; reply with the selected email only.",
    model = "gpt-4o-mini"
)


In [15]:

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [22]:
sales_agent1 = Agent(
    name = "Professional sales Agent",
    instructions = instruction1,
    model = "gpt-4o-mini"
)
sales_agent2 = Agent(
    name = "Professional sales Agent",
    instructions = instructions2,
    model = "gpt-4o-mini"
)
sales_agent3 = Agent(
    name = "Professional sales Agent",
    instructions = instructions3,
    model = "gpt-4o-mini"
)

In [23]:
message = "Write a cold sales email"

with trace("pick the best email"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )


outputs = [result.final_output for result in results]

In [25]:
 emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

In [27]:
best = await Runner.run(sales_picker, emails)

In [28]:
best

RunResult(input="Cold sales emails:\n\nSubject: Simplify Your SOC 2 Compliance with ComplAI\n\nHi [Recipient's Name],\n\nI hope this message finds you well.\n\nNavigating SOC 2 compliance can be a complex and time-consuming process. At ComplAI, we understand the challenges companies face when preparing for audits and ensuring compliance. That’s why we've developed an AI-powered SaaS tool designed to streamline your compliance efforts and enhance your audit readiness.\n\n**Key Benefits of Using ComplAI:**\n\n- **Automated Documentation:** Our platform simplifies the documentation process, providing templates and guidance to help you stay compliant with minimal effort.\n- **Real-Time Monitoring:** Keep track of your compliance status effortlessly, with real-time updates and alerts to ensure you're always prepared for your next audit.\n- **Expert Insights:** Gain access to our library of resources and expert advice tailored to your industry's specific compliance requirements.\n\nMany orga

In [29]:
best.final_output

"Subject: Your SOC2 Compliance Journey Just Got a Makeover! ✨\n\nHi [Recipient's Name],\n\nI hope this email finds you wading through the paperwork with a smile (or a caffeinated beverage, at least)! ☕ \n\nI’m reaching out from ComplAI, where we believe that getting SOC2 compliant shouldn’t feel like climbing Everest—more like a stroll through a park, complete with a barbecue. 🌳🔥 \n\nOur AI-powered tool does the heavy lifting, handling everything from documentation to audit prep. Think of it as your trusty sidekick, but instead of a cape, it comes with compliance magic! 🦸\u200d♂️✨\n\nImagine being able to check “SOC2 compliance” off your to-do list without breaking a sweat. Sounds dreamy, right? 😴💭 \n\nLet’s schedule a quick call! I promise, no legalese or jargon—just some good ol’ common sense and a sprinkle of humor. \n\nLooking forward to hearing from you!\n\nBest,  \n[Your Name]  \nComplAI – Making Compliance Less of a Compliance-tion!  \n[Your Contact Information]  \n[Your LinkedI

### tools and agent interaction

Wrap the function with the decorator @`function_tool
`


Convert a function into a tool


In [42]:
@function_tool # covented inot ools
def send_email(body:str):
    send_grid = sendgrid.SendGridAPIClient(api_key=os.getenv('SENDGRID_API_KEY'))
    from_email = Email('dpoorna1995@gmail.com')
    to_email = To('poorna7d@gmail.com')
    content = Content("text/plain", body)
    mail = Mail(from_email, to_emails, "sales email", content).get()
    response = send_grid.mail.send.post(request_body = mail)
    return {"status":'sucess'}

In [43]:

send_email



FunctionTool(name='send_email', description='', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10dd20cc0>, strict_json_schema=True, is_enabled=True)

### convert a agent into a tool

In [33]:
tool1 = sales_agent1.as_tool(tool_name= "sales_agent1", tool_description="write an cold sales emails")
tool1

FunctionTool(name='sales_agent1', description='write an cold sales emails', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10eddefc0>, strict_json_schema=True, is_enabled=True)

#### bring all together

a tool for all out 3 email writing agents
and a tool for our funtion to send emails


In [44]:
description = "write an cold sales emails"

tool1 = sales_agent1.as_tool(tool_name= "sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name= "sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name= "sales_agent3", tool_description=description)


tools = [tool1, tool2, tool3, send_email]


In [35]:
tools

[FunctionTool(name='sales_agent1', description='write an cold sales emails', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10d7c14e0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='write an cold sales emails', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10d7c18a0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='write an cold sales emails', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'req

sales manager - our palnning agent




In [45]:

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""

In [46]:
instructions


'\nYou are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.\n\nFollow these steps carefully:\n1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.\n\n2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.\n\n3. Use the send_email tool to send the best email (and only the best email) to the user.\n\nCrucial Rules:\n- You must use the sales agent tools to generate the drafts — do not write them yourself.\n- You must send ONE email using the send_email tool — never more than one.\n'

In [47]:
sales_manager = Agent(name = "Sales Manager", instructions= instructions, tools = tools, model = "gpt-4o-mini")

message = "send a cold sales adreeses to dear ceo"

In [48]:
with trace("Sales Manager"):
    result = await Runner.run(sales_manager, message )

In [49]:
result.final_output

'It seems there was an issue while trying to send the email. I can help you draft a final email to send to the CEO. Would you like me to help you with that? If you have any specific preferences for the content, please let me know!'

### handsoff reprsenr a way an agent can delegeate to an agent , passing controll to it

new agent


In [50]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [53]:
@function_tool
def send_html_email(subject:str, html_body:str) ->Dict[str,str]:
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get("SENDGRID_API_KEY"))
    from_email = Email("dpoorna1995@gmail.com")
    to_email = to("poorna7d@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email,to_email,content).get()
    sg.client.mail.send.post(request_body = mail)
    return {"status":"sucess"}

In [54]:
tools = [subject_tool, html_tool,send_html_email ]

In [55]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10dd22de0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10dd237e0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='', params_json_schema={'properties': {'subject': {'title': 'Subject', 't

In [56]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


In [58]:
email_sender = Agent(
    name= "Email Manager",
    instructions = instructions,
    tools = tools,
    model = "gpt-4o-mini",
    handoff_description = "convert an emal to HTML and send it"
)

####  Now we have 3 tools and 1 handoff

In [61]:
tools = [tool1, tool2,tool3]
handoffs = [email_sender]

print(tools)

[FunctionTool(name='sales_agent1', description='write an cold sales emails', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10dd211c0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='write an cold sales emails', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10dd21580>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='write an cold sales emails', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'requi

In [60]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""

In [62]:
sales_manager = Agent(
    name = "Sales Manager",
    instructions = sales_manager_instructions,
    tools = tools,
    handoffs = handoffs,
    model = "gpt-4o-mini"
)
message = " send out an cold sales eail adress to Dear CEO"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

In [68]:

result

RunResult(input=' send out an cold sales eail adress to Dear CEO', new_items=[ToolCallItem(agent=Agent(name='Sales Manager', handoff_description=None, tools=[FunctionTool(name='sales_agent1', description='write an cold sales emails', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10dd211c0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='write an cold sales emails', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10dd21580>, strict_json_schema=True, is_enabled=True), Function